# Introduction

This notebook demonstrates how to perform structure enumeration using Python Materials Genomics (pymatgen). These notebooks are described in detail in

    Deng, Z.; Zhu, Z.; Chu, I.-H.; Ong, S. P. Data-Driven First-Principles Methods for the Study and Design of 
    Alkali Superionic Conductors. Chem. Mater. 2017, 29 (1), 281–288 DOI: 10.1021/acs.chemmater.6b02648.
    
If you find these notebooks useful and use the functionality demonstrated, please consider citing the above work.

Let's start by importing some modules and classes that we will be using.

In [ ]:
# Uncomment the subsequent lines in this cell to install dependencies for Google Colab.
# !pip install pymatgen==2022.7.19

In [ ]:
from pymatgen.core import Structure
from pymatgen.io.vasp.sets import MPRelaxSet, batch_write_input
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.transformations.advanced_transformations import (
    EnumerateStructureTransformation,
)

# Preparation

We will first read in the structure from crystallographic information file (CIF) found in ICSD.

In [ ]:
structure = Structure.from_file("aimd_data/EntryWithCollCode418490.cif")
print(structure)

Full Formula (Li26.88 P4 S20 Cl4)
Reduced Formula: Li26.88P4S20Cl4
abc   :   9.859000   9.859000   9.859000
angles:  90.000000  90.000000  90.000000
Sites (76)
  #  SP               a        b        c
---  ---------  -------  -------  -------
  0  Li+:0.560  0.3148   0.982    0.3148
  1  Li+:0.560  0.982    0.6852   0.6852
  2  Li+:0.560  0.6852   0.3148   0.018
  3  Li+:0.560  0.3148   0.6852   0.018
  4  Li+:0.560  0.982    0.3148   0.3148
  5  Li+:0.560  0.6852   0.982    0.6852
  6  Li+:0.560  0.3148   0.018    0.6852
  7  Li+:0.560  0.6852   0.6852   0.982
  8  Li+:0.560  0.3148   0.3148   0.982
  9  Li+:0.560  0.6852   0.018    0.3148
 10  Li+:0.560  0.018    0.6852   0.3148
 11  Li+:0.560  0.018    0.3148   0.6852
 12  Li+:0.560  0.3148   0.482    0.8148
 13  Li+:0.560  0.982    0.1852   0.1852
 14  Li+:0.560  0.6852   0.8148   0.518
 15  Li+:0.560  0.3148   0.1852   0.518
 16  Li+:0.560  0.982    0.8148   0.8148
 17  Li+:0.560  0.6852   0.482    0.1852
 18  Li+:0.560  0.3148  

From the above, we see that the reported experimental structure has Li disorder. The occupancy of Li is higher than what would be expected for a Li6PS5Cl nominal composition. We will first adjust the composition by setting the Li occupancies to 0.5 to obtain stoichiometric charge-balanced Li6PS5Cl.

In [ ]:
# loop over all sites in the structure
for i, site in enumerate(structure):
    # change the occupancy of Li+ disordered sites to 0.5
    if not site.is_ordered:
        structure[i] = {"Li+": 0.5}
print(
    "The composition after adjustments is %s." % structure.composition.reduced_formula
)

The composition after adjustments is Li6PS5Cl.


To keep the number of orderings manageable, we will perform enumeration only on the primitive cell. The primitive cell can be obtained using the *SpacegroupAnalyzer*.

In [ ]:
analyzer = SpacegroupAnalyzer(structure)
prim_cell = analyzer.find_primitive()
print(prim_cell)

Full Formula (Li6 P1 S5 Cl1)
Reduced Formula: Li6PS5Cl
abc   :   6.971366   6.971366   6.971366
angles:  60.000000  60.000000  60.000000
Sites (19)
  #  SP               a        b        c
---  ---------  -------  -------  -------
  0  Li+:0.500  0.3616   0.768    0.768
  1  Li+:0.500  0.1024   0.768    0.3616
  2  Li+:0.500  0.768    0.768    0.1024
  3  Li+:0.500  0.768    0.768    0.3616
  4  Li+:0.500  0.3616   0.768    0.1024
  5  Li+:0.500  0.1024   0.768    0.768
  6  Li+:0.500  0.768    0.1024   0.3616
  7  Li+:0.500  0.1024   0.3616   0.768
  8  Li+:0.500  0.3616   0.1024   0.768
  9  Li+:0.500  0.768    0.3616   0.1024
 10  Li+:0.500  0.768    0.3616   0.768
 11  Li+:0.500  0.768    0.1024   0.768
 12  P5+        0.25     0.25     0.25
 13  S2-        0        0        0
 14  S2-        0.36947  0.36947  0.89159
 15  S2-        0.36947  0.36947  0.36947
 16  S2-        0.89159  0.36947  0.36947
 17  S2-        0.36947  0.89159  0.36947
 18  Cl-        0.75     0.75     0.75


# Enumerate structures using enumlib

We will use the *EnumerateStructureTransformation* class to enumerate all symmetrically distinct structures. *EnumerateStructureTransformation* is a user-friendly wrapper around enumlib, a fortran library to generate derivative structures written by Hart et al.

In [ ]:
enum = EnumerateStructureTransformation()
enumerated = enum.apply_transformation(
    prim_cell, 100
)  # return no more than 100 structures
structures = [d["structure"] for d in enumerated]
print("%d structures returned." % len(structures))

48 structures returned.


# VASP input generation

Pymatgen has useful classes for batch generating VASP input files that use parameters that are compatible with those used in the Materials Project. These parameters have been well-tested over a large database of structures in different chemistries. Using the same parameters also allow the energies computed to be compared with those in the Materials Project database for phase stability and other analyses.

In [ ]:
batch_write_input(
    structures, vasp_input_set=MPRelaxSet, output_dir="Li6PS5Cl_orderings"
)